In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# ChatOpenAI 모델을 생성합니다. 'temperature'는 모델의 응답 다양성을 조절합니다.
llm = ChatOpenAI(
    temperature=0.1,  # 낮은 값으로 설정하여 응답의 일관성을 유지
)

# 로컬 캐시 저장소를 생성합니다. 캐시는 임베딩 저장을 위한 디렉토리를 정의합니다.
cache_dir = LocalFileStore("./.cache/")

# 텍스트를 분할하기 위한 TextSplitter를 생성합니다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # 줄바꿈 문자로 텍스트를 분할
    chunk_size=600,  # 각 청크의 최대 문자 수
    chunk_overlap=100,  # 청크 간 중복 문자 수
)

# 텍스트 파일을 로드합니다. 여기서는 "chapter_one.txt" 파일을 로드합니다.
loader = UnstructuredFileLoader("./document.txt")

# 로드한 파일을 정의된 TextSplitter를 사용하여 분할합니다.
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩 모델을 초기화합니다.
embeddings = OpenAIEmbeddings()

# 캐시를 지원하는 임베딩 객체를 생성합니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 분할된 문서를 기반으로 FAISS 벡터 스토어를 생성합니다.
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 벡터 스토어를 검색 도구로 변환합니다.
# chain의 가장 중요한 구성요소
retriver = vectorstore.as_retriever()

# 채팅 프롬프트 템플릿을 정의합니다. 컨텍스트 기반 응답을 생성하도록 구성합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),  # 사용자 질문을 동적으로 삽입
    ]
)

# 최종 질의응답 체인을 구성합니다.
chain = (
    {
        "context": retriver,  # 컨텍스트를 검색합니다.
        "question": RunnablePassthrough(),  # 질문을 그대로 전달합니다.
    }
    | prompt  # 프롬프트 템플릿에 데이터를 전달합니다.
    | llm  # LLM(ChatOpenAI)을 사용하여 응답을 생성합니다.
)

# 체인을 실행하여 질문에 답변을 생성합니다.
response = chain.invoke("Describe Victory Mansions")
print(response)


content='Victory Mansions is a place where Winston lives. It is a run-down apartment building with small, shabby units. The building is in disrepair, with faulty plumbing and electricity. The atmosphere is grim and oppressive, reflecting the overall bleakness of the society in which Winston resides.'
